# Business Analytics

We have provided a sample of 460 games (230 from the 2016-17 season and 230 from the 2017-18 season) for which your task is to predict total international viewers. To assist with your model, we have provided with viewership numbers from 1,000 games by country in each of the two seasons, plus stats on team and player performance and status. 

#### Using these inputs, we would like you to predict the total number of international viewers for each of the 460 games in the test set.

Team game stats and player stats are provided for all games in the training data set, but have been removed from the games in the test set.

You will be graded on Mean Absolute Percentage Error (MAPE) on Total Viewers. We selected this metric due to natural scaling in the international viewership data. This metric is defined as:

\begin{equation*}
MAPE = \frac{1}{n} \sum_{i=1}^n \lvert \frac{A_i-P_i}{A_i}\rvert
\end{equation*}

where 𝑛 = 460 is the total observations in the test set, and 𝐴𝑖 and 𝑃𝑖 are the 𝑖th actual and predicted Total Viewers. Some tips to help you in your modeling:
- Consider all factors that may drive viewership. Team strength may be one, but there may be others such as market size and/or social following.
- Consider temporal/seasonal effects such as day of week, opening day/week and holidays.
- Consider using other public information like Google Trends or further historical team performance if you find it may helpful. Not required.

Please return to us a copy of test\_set_[Team_Name].csv with the “Total Viewers” column filled in with your response variable. Please also return all code or relevant working files. Thank you!

### Import Packages

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor

### Load 'game_data.csv'

This dataset includes incoming wins and losses (i.e., record going into a game), dates, and selected game stats for each game in the 2016-17 and 2017-18 seasons. Game stats are only included for games in the training set – not in the test set.

In [2]:
Game_Data = pd.read_csv('game_data.csv', sep=',', header = 0)

# Turn Game_Date string to datetime
Game_Data.Game_Date = pd.to_datetime(Game_Data.Game_Date, format="%m/%d/%Y")

Game_Data.head()

,Season,Game_ID,Game_Date,Team,Location,Wins_Entering_Gm,Losses_Entering_Gm,Team_Minutes,Final_Score,Lead_Changes,Ties,Largest_Lead,Full_Timeouts,Short_Timeouts,Qtr_4_Score,L2M_Score
0,2016-17,21600001,2016-10-25,CLE,H,0,0,240.0,117.0,1.0,3.0,32.0,4.0,1.0,82.0,113.0
1,2016-17,21600001,2016-10-25,NYK,A,0,0,240.0,88.0,0.0,1.0,4.0,4.0,2.0,64.0,84.0
2,2016-17,21600002,2016-10-25,POR,H,0,0,240.0,113.0,6.0,6.0,13.0,5.0,0.0,77.0,104.0
3,2016-17,21600002,2016-10-25,UTA,A,0,0,240.0,104.0,5.0,1.0,8.0,5.0,2.0,83.0,102.0
4,2016-17,21600003,2016-10-25,GSW,H,0,0,240.0,100.0,1.0,2.0,4.0,3.0,1.0,77.0,100.0


### Load 'player_data.csv'

This dataset includes performance stats for each player in each game in the 2016-17 and 2017-18 seasons. It also includes indicators for whether the player was selected as an All-Star for the season in question and whether the player was active for the game. Game statistics are only included for games in the training set – not in the test set. All-Star status and active vs. inactive status is provided for all games.

In [3]:
Player_Data = pd.read_csv('player_data.csv', sep=',', header = 0)

# Turn Game_Date string to datetime
Player_Data.Game_Date = pd.to_datetime(Player_Data.Game_Date, format="%m/%d/%Y")

Player_Data.head()

,Season,Game_ID,Game_Date,Team,Person_ID,Name,ASG_Team,Active_Status,Minutes,Points,...,Steals,Blocks,Turnovers,Field_Goals,Field_Goals_Attempted,Three_Pointers,Three_Pointers_Attempted,Free_Throws,Free_Throws_Attempted,Personal_Fouls
0,2016-17,21600001,2016-10-25,CLE,101112,Channing Frye,None,Active,14.0,6.0,...,1.0,1.0,1.0,2.0,6.0,2.0,5.0,0.0,0.0,4.0
1,2016-17,21600001,2016-10-25,CLE,201567,Kevin Love,East ASG,Active,25.0,23.0,...,3.0,0.0,2.0,6.0,15.0,2.0,6.0,9.0,12.0,3.0
2,2016-17,21600001,2016-10-25,CLE,202681,Kyrie Irving,East ASG,Active,30.0,29.0,...,2.0,0.0,0.0,12.0,22.0,4.0,7.0,1.0,1.0,1.0
3,2016-17,21600001,2016-10-25,CLE,202684,Tristan Thompson,None,Active,22.0,0.0,...,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0
4,2016-17,21600001,2016-10-25,CLE,202697,Iman Shumpert,None,Active,14.0,2.0,...,0.0,1.0,2.0,1.0,3.0,0.0,1.0,0.0,0.0,2.0


### Load 'training_set.csv'

This dataset includes total viewership for each international country for each game in the 2016-17 and 2017-18 seasons. Included are 1,000 games from each season.

In [4]:
Training_Set = pd.read_csv('training_set.csv', sep=',', header = 0)

# Turn Game_Date string to datetime
Training_Set.Game_Date = pd.to_datetime(Training_Set.Game_Date, format="%m/%d/%Y")

Training_Set.head()

,Season,Game_ID,Game_Date,Away_Team,Home_Team,Country,Rounded Viewers
0,2016-17,21600001,2016-10-25,NYK,CLE,C113,18
1,2016-17,21600001,2016-10-25,NYK,CLE,C193,1
2,2016-17,21600001,2016-10-25,NYK,CLE,C126,4
3,2016-17,21600001,2016-10-25,NYK,CLE,C163,11
4,2016-17,21600001,2016-10-25,NYK,CLE,C73,3


### Load 'test_set.csv'

This dataset includes a list of games in the 2016-17 and 2017-18 seasons for which you are expected to predict total international viewership (i.e., the sum across all countries). Included are 230 games from each season.

In [5]:
Test_Set = pd.read_csv('test_set.csv', sep=',', header = 0)
Test_Set.head()

,Season,Game_ID,Game_Date,Away_Team,Home_Team,Total_Viewers
0,2016-17,21600010,10/26/2016,DEN,NOP,NaN
1,2016-17,21600023,10/28/2016,HOU,DAL,NaN
2,2016-17,21600027,10/29/2016,BOS,CHA,NaN
3,2016-17,21600026,10/29/2016,ATL,PHI,NaN
4,2016-17,21600048,11/1/2016,LAL,IND,NaN


### Create Helper functions:

In [6]:
def create_data(game_data, player_data, type, season):

    '''Given game and player data, returns dataset with all variables'''
    
    # create data
    if type == 'train':
        data = Training_Set[Training_Set.Season==season].groupby('Game_ID').sum().reset_index()
        data = data.rename(index=str, columns={'Rounded Viewers': 'Total_Viewers'})
    elif type == 'test':
        data = Test_Set[Test_Set.Season==season].groupby('Game_ID').sum().reset_index()
    
    # Add home team, home W, home L, date, day
    data = data.merge(game_data[game_data.Location=='H'][['Game_ID', 'Game_Date', 'Team', 'Wins_Entering_Gm', 'Losses_Entering_Gm']], on='Game_ID')
    data = data.rename(index=str, columns={'Team': 'Home_Team', 'Wins_Entering_Gm': 'Home_W', 'Losses_Entering_Gm': 'Home_L'})
    data['Day'] = data.Game_Date.dt.dayofweek
    
    # Add away team, away W, away L
    data = data.merge(game_data[game_data.Location=='A'][['Game_ID', 'Team', 'Wins_Entering_Gm', 'Losses_Entering_Gm']], on='Game_ID')
    data = data.rename(index=str, columns={"Team": "Away_Team", 'Wins_Entering_Gm': 'Away_W', 'Losses_Entering_Gm': 'Away_L'})
    
    # Add number of all stars playing
    player_data.ASG_Team = player_data.ASG_Team.mask(player_data.ASG_Team == 'None', 0).mask(player_data.ASG_Team != 'None', 1)
    asg = player_data.groupby('Game_ID')['ASG_Team'].sum().reset_index()
    data = data.merge(asg[['Game_ID', 'ASG_Team']], on="Game_ID")
    
    #Add CSA data
    # from https://www.reddit.com/r/nba/comments/4ripnx/lets_figure_out_actual_market_sizes/
    csa_dict = {'Home_Team': ['NYK','BKN','LAL','LAC','CHI','WAS','GSW','BOS','DAL','PHI','HOU','MIA','ATL','TOR','DET','PHX','MIN','CLE','DEN',
              'ORL','POR','CHA','SAC','UTA','SAS','IND','MIL','NOP','OKC','MEM'], 
     'Away_Team': ['NYK','BKN','LAL','LAC','CHI','WAS','GSW','BOS','DAL','PHI','HOU','MIA','ATL','TOR','DET','PHX','MIN','CLE','DEN',
              'ORL','POR','CHA','SAC','UTA','SAS','IND','MIL','NOP','OKC','MEM'], 
     'CSA': [23723696,23723696,18679763,18679763,9923358,9625360,8713914,8152573,7504362,7183479,6855069,6654565,
             6365108,6054191,5319913,4574531,3866768,3493596,3418876,3129308,3110906,2583956,2544026,2467709,2384075,
             2372530,2046092,1493205,1430327,1370716]}
    csa_data = pd.DataFrame(data=csa_dict)
    data = data.merge(csa_data[['Home_Team', 'CSA']], on='Home_Team')
    data = data.rename(index=str, columns={'CSA' : 'Home_CSA'})
    data = data.merge(csa_data[['Away_Team', 'CSA']], on='Away_Team')
    data = data.rename(index=str, columns={'CSA' : 'Away_CSA'})

    # Add franchise data
    # from https://www.statista.com/statistics/193696/franchise-value-of-national-basketball-association-teams-in-2010/
    franchise_dict = {'Home_Team': ['NYK','BKN','LAL','LAC','CHI','WAS','GSW','BOS','DAL','PHI','HOU','MIA','ATL','TOR','DET','PHX','MIN','CLE','DEN',
              'ORL','POR','CHA','SAC','UTA','SAS','IND','MIL','NOP','OKC','MEM'], 
     'Away_Team': ['NYK','BKN','LAL','LAC','CHI','WAS','GSW','BOS','DAL','PHI','HOU','MIA','ATL','TOR','DET','PHX','MIN','CLE','DEN',
              'ORL','POR','CHA','SAC','UTA','SAS','IND','MIL','NOP','OKC','MEM'], 
     'Franchise': [3600,2300,3300,2150,2600,1350,3100,2500,1900,1180,2200,1700,1150,1400,1100,1280,1060,1325,1125,1225,1300,1050,1375,1200,1550,1175,1075,1000,1250,1025]}
    franchise_data = pd.DataFrame(data=franchise_dict)
    data = data.merge(franchise_data[['Home_Team', 'Franchise']], on='Home_Team')
    data = data.rename(index=str, columns={'Franchise' : 'Home_Franchise'})
    data = data.merge(franchise_data[['Away_Team', 'Franchise']], on='Away_Team')
    data = data.rename(index=str, columns={'Franchise' : 'Away_Franchise'})
    
    # Add average viewers data
    avg_viewers = Game_Data[['Game_ID', 'Team']].merge(Training_Set.groupby(['Game_ID', 'Home_Team', 'Away_Team']).sum().reset_index(), on='Game_ID')
    home_avg_viewers = avg_viewers.groupby('Team').mean().reset_index().rename(index=str, columns={'Team' : 'Home_Team', 'Rounded Viewers': 'Home_Avg'})[['Home_Team', 'Home_Avg']]
    away_avg_viewers = avg_viewers.groupby('Team').mean().reset_index().rename(index=str, columns={'Team' : 'Away_Team', 'Rounded Viewers': 'Away_Avg'})[['Away_Team', 'Away_Avg']]
    data = data.merge(home_avg_viewers, on='Home_Team')
    data = data.merge(away_avg_viewers, on='Away_Team')
    
    # Return data
    data = data[['Game_ID', 'Day', 'Game_Date', 'Home_W',
       'Home_L', 'Away_W', 'Away_L', 'ASG_Team',
       'Home_CSA', 'Away_CSA', 'Home_Franchise', 'Away_Franchise', 'Home_Avg', 'Away_Avg', 'Total_Viewers',]]
    return data

In [7]:
def randomforest(X_train, X_test, Y_train, Y_test):
    '''Use random forest regression as model'''
    randomforest = RandomForestRegressor(random_state = 0, n_estimators=20, min_samples_leaf= 3)
    randomforest.fit(X_train, Y_train)
    randomforest_prediction = randomforest.predict(X_test)
    if type(Y_test) == type('none'):
        return randomforest_prediction
    else:
        mape = np.mean(np.abs((Y_test - randomforest_prediction) / Y_test)) * 100
        return randomforest_prediction, mape

In [8]:
def evaluate_model(data):
    
    '''Evaluates the model using MAPE calculation'''
    
    # use K-fold cross validation to evaluate our model
    nsplits = 10
    kf = KFold(n_splits=nsplits, shuffle=True)
    mape_array = np.zeros(nsplits)
    n = 0
        
    for train_index, test_index in kf.split(data):
        X = data[['ASG_Team', 'Day', 'Home_W', 'Home_L', 'Away_W', 'Away_L', 'Home_CSA', 'Away_CSA', 
                  'Home_Franchise', 'Away_Franchise', 'Home_Avg', 'Away_Avg']]
        Y = data[['Total_Viewers']]
        X_train, X_test = X.iloc[train_index].reset_index(), X.iloc[test_index].reset_index()
        Y_train, Y_test = Y.iloc[train_index].reset_index()['Total_Viewers'], Y.iloc[test_index].reset_index()['Total_Viewers']
        pred, mape = randomforest(X_train, X_test, Y_train, Y_test)
        mape_array[n] = mape
        n = n + 1
        print('Split #' + str(n) + ': ' + str(mape))

    print('Avg: ' + str(np.mean(mape_array)))

### Perform Analysis

Different seasons should have different models based on different data.

In [9]:
# Separate into 2016-2017 and 2017-2018 data
data1617 = create_data(Game_Data[Game_Data.Season == '2016-17'].dropna(), Player_Data[Player_Data.Season == '2016-17'].dropna(), 'train', '2016-17')
data1718 = create_data(Game_Data[Game_Data.Season == '2017-18'].dropna(), Player_Data[Player_Data.Season == '2017-18'].dropna(), 'train', '2017-18')

Using K-fold cross validation, evaluate how good the model is for each season.

In [10]:
# Evaluate the model
evaluate_model(data1617)

Split #1: 21.1361647171
Split #2: 25.8057376586
Split #3: 28.1161328725
Split #4: 37.6520428739
Split #5: 30.7271404548
Split #6: 31.4862994779
Split #7: 28.9094595174
Split #8: 27.2917482437
Split #9: 24.0891237863
Split #10: 33.7441144323
Avg: 28.8957964034


In [11]:
# Evaluate the model
evaluate_model(data1718)

Split #1: 26.9359298695
Split #2: 29.4856878059
Split #3: 35.9753949965
Split #4: 27.5400711696
Split #5: 35.6837899581
Split #6: 32.8530605446
Split #7: 33.2817237798
Split #8: 29.5811528578
Split #9: 29.7519570944
Split #10: 28.8792084647
Avg: 30.9967976541


Create the datasets needed for predicting values belonging in the Test_Set.

In [12]:
# Create training data, by season
X_1617 = data1617[['ASG_Team', 'Day', 'Home_W', 'Home_L', 'Away_W', 'Away_L', 'Home_CSA', 'Away_CSA', 'Home_Franchise', 'Away_Franchise']]
Y_1617 = data1617[['Total_Viewers']]
X_1718 = data1718[['ASG_Team', 'Day', 'Home_W', 'Home_L', 'Away_W', 'Away_L', 'Home_CSA', 'Away_CSA', 'Home_Franchise', 'Away_Franchise']]
Y_1718 = data1718[['Total_Viewers']]

# Create test data, by season
data_test_1617 = create_data(Game_Data, Player_Data, 'test', '2016-17')
data_test_1718 = create_data(Game_Data, Player_Data, 'test', '2017-18')
X_test_1617 = data_test_1617[['ASG_Team', 'Day', 'Home_W', 'Home_L', 'Away_W', 'Away_L', 'Home_CSA', 'Away_CSA', 'Home_Franchise', 'Away_Franchise']]
X_test_1718 = data_test_1718[['ASG_Team', 'Day', 'Home_W', 'Home_L', 'Away_W', 'Away_L', 'Home_CSA', 'Away_CSA', 'Home_Franchise', 'Away_Franchise']]

Fill in the predictions based on these datasets.

In [13]:
# Fill in Test_Set based on model
Test_Set.loc[Test_Set.Season=='2016-17', 'Total_Viewers'] = randomforest(X_1617, X_test_1617, Y_1617.values.ravel(), 'none')
Test_Set.loc[Test_Set.Season=='2017-18', 'Total_Viewers'] = randomforest(X_1718, X_test_1718, Y_1718.values.ravel(), 'none')
Test_Set.Total_Viewers = Test_Set.Total_Viewers.round().astype(int)

Save to .csv

In [14]:
# Save to .csv
Test_Set.to_csv('test_set_airballs.csv', index=False)